In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [ ]:
data_path = r"C:\Users\LENOVO\Desktop\Python\Datasets\Alzeimehers\Data"

data_gen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.2,
                             horizontal_flip=True,
                             zoom_range=0.2)

train_generator = data_gen.flow_from_directory(data_path,
                                              target_size=(224, 224),
                                              batch_size=128,
                                              class_mode='binary',
                                              subset='training')

val_generator = data_gen.flow_from_directory(data_path,
                                            target_size=(224, 224),
                                            batch_size=128,
                                            class_mode='binary',
                                            subset='validation')


Found 14982 images belonging to 2 classes.
Found 3745 images belonging to 2 classes.


In [ ]:
print(train_generator.class_indices)

{'Mild_Dementia': 0, 'Very_mild_Dementia': 1}


In [ ]:
class_indices = train_generator.class_indices
classes = list(class_indices.values())

labels = train_generator.classes

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weight_dict = dict(zip(classes, class_weights))

print(class_weight_dict)

{0: np.float64(1.8718140929535232), 1: np.float64(0.6822404371584699)}


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 17s 0us/step


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy', 'precision', 'recall'])

In [ ]:
history = model.fit(train_generator,
          validation_data=val_generator,
          epochs=10,)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train loss', 'val loss'])
plt.title('Loss per Epoch');

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'])
plt.title('Accuracy');

# Description of the Model  
The model is built using transfer learning by leveraging the VGG16 architecture pretrained on the ImageNet dataset.   

Key aspects:   
VGG16 Base: The convolutional layers of VGG16 are used as a fixed feature extractor (trainable=False).

Custom Top Layers:  
Flatten layer to convert feature maps to a vector.   
Dense(256) fully connected layer with ReLU activation.   
Dropout(0.5) to prevent overfitting.   
Dense(1) output layer with sigmoid activation for binary classification (Alzheimer’s vs Healthy).

Loss Function:   
binary_crossentropy, suitable for binary classification.  
Optimizer:   
Adam with a learning rate of 1e-4.  
Metrics: Accuracy, Precision, and Recall are tracked.

# Description of the Code   
ImageDataGenerator is used for:   
Rescaling pixel values to [0, 1]   
Data augmentation (horizontal flip and zoom)   
Splitting data into training (80%) and validation (20%) subsets.   
Class Weights are computed using sklearn.utils.compute_class_weight to handle class imbalance.

Transfer Learning:   
VGG16 is loaded with pretrained ImageNet weights (excluding top classification layers).   
All convolutional layers are frozen.  
Custom dense layers are added for classification.

Model Compilation:   
Optimizer: Adam   
Loss: binary_crossentropy   
Metrics: accuracy, precision, recall.

Model Training:   
Trained for 10 epochs with the augmented data generators.

Performance Visualization:       
Loss and accuracy plots are generated for both training and validation sets across epochs.

Performance   
After training for 10 epochs:  
Training and Validation Accuracy trends are visualized.  
Loss and Accuracy Curves are plotted:

Helps detect overfitting/underfitting.   
Shows how well the model is learning over time.   
(Actual numeric values aren't shown here since they're runtime dependent, but your code correctly tracks and plots them.)

# My Comments
Using pretrained Architectures can significantly Increase Model performance.   
Using transfer learning saves a lot of time and resources that would have been spent on model training and architecture development